# Naive Bayes

Bir spam algılama sorunu için Naive Bayes algoritmasını uygulayacaz. 1 - 3. Bölümler sorun hakkında yararlı bağlamlar sağlar. 4. Bölümde algoritmayı gerçekten uygulayacak işlevler yazacaz. 5. Bölümde bazı ilginç notlandırılmamış uzantılar yer almaktadır.

# Anahat
- [ 1 - Giriş](#1)
- [ 2 - Gerekli içe aktarmalar](#2)
- [ 3 - Veri Kümesi](#3)
  - [ 3.1 Veri Kümesini Yükleme ve İnceleme](#3.1)
  - [ 3.2 Veri Kümesini Ön İşleme](#3.2)
  - [ 3.3 Metni Ön İşleme](#3.3)
  - [ 3.4 Eğitim/test olarak bölme](#3.4)
- [ 4 - Naive Bayes Algoritmasını Uygulama](#4)
  - [ 4.1 $P(\text{email} \mid \text{spam})$ ve $P(\text{email} \mid \text{ham})$ Hesaplama](#4.1)
  - [ 4.2 $P(\text{spam})$ ve $P(\text{ham})$ Hesaplama](#4.2)

  - [ 4.3 Her şeyi bir araya getirme](#4.3)
    - [ Alıştırma 1](#ex01)
    - [ Alıştırma 2](#ex02)
    - [ Alıştırma 3](#ex03)
    - [ Alıştırma 4](#ex04)
  - [ 4.4 Model performansı](#4.4)
- [ 5 - Ek Bölüm](#5)
  - [ 5.1 Naive Bayes modelindeki gizli sorun.](#5.1)
  - [ 5.2 Model performansını geliştirme: Naive Bayes ile pratik uygulama](#5.2)

<a name="1"></a>
## 1 - Giriş

Naive Bayes algoritması, bir veri noktasının belirli bir sınıfa ait olup olmadığını belirleme amacıyla Bayes Teoremi'nden yararlanarak Makine Öğrenimi ve Veri Bilimi'nde bir temel taşı olarak durmaktadır. Algoritma, her bir özelliğin diğerlerinden bağımsız olduğu yönünde "naive bir varsayım" yapar. Bu varsayım, verileriniz için neredeyse kesinlikle doğru değildir, ancak bunu yapmak, uygulanması önemli ölçüde daha kolay bir algoritmaya yol açar ve göreceğiniz gibi, etkileyici derecede faydalı sonuçlara yol açabilir. Naive Bayes'in denetlenen bir algoritma olduğunu, yani etkili bir şekilde çalışması için önceden etiketlenmiş verilere ihtiyaç duyduğunu belirtmek önemlidir. Göreceğiniz örnekte, algoritmayı eğitmek için bir dizi e-postanın önceden "spam" veya "ham" olarak işaretlenmiş olması gerektiği anlamına gelir.

### Spam Algılama için Naive Bayes

Bu ödev ikili bir sınıflandırma problemine odaklanmaktadır: spam ve spam olmayan e-postalar arasında ayrım yapmak, halk arasında "ham" olarak anılır. Bu görevin amacı için, spam e-postalar $1$ olarak etiketlenecek ve spam olmayan (ham) e-postalar $0$ olarak etiketlenecektir.

Belirli bir e-posta için ilgi olasılığı şu şekilde gösterilir:

$$ P(\text{spam} \mid \text{email}) $$

Bu olasılık ne kadar yüksekse e-postanın spam olarak sınıflandırılma olasılığı da o kadar yüksektir. Bayes Teoremi hesaplamada şu şekilde kullanılır:

$$ P(\text{spam} \mid \text{email}) = \frac{P(\text{email} \mid \text{spam}) \cdot P(\text{spam})}{P(\text{email})} $$

Terimlerin dökümü şöyle:

- $ P(\text{spam}) $: Rastgele seçilen bir e-postanın spam olma olasılığı, veri kümesindeki spam e-postaların oranına eşittir.
- $ P(\text{email} \mid \text{spam}) $: Spam olduğu bilinen belirli bir e-postanın oluşma olasılığı.
- $ P(\text{email}) $: E-postanın oluşma genel olasılığı.

Bu yaklaşımda kullanabileceğiniz ilginç bir erken "kısayol" $ P(\text{email}) $ terimini görmezden gelmektir. Bu hesaplamanın amacı bir e-postanın spam olma olasılığını ham olma olasılığıyla karşılaştırmaktır. İşte hem $ P(\text{spam} \mid \text{email}) $ hem de $ P(\text{ham} \mid \text{email}) $ için ifadeler:

$$ P(\text{spam} \mid \text{email}) = \frac{P(\text{email} \mid \text{spam}) \cdot P(\text{spam})}{P(\text{email})} $$

$$ P(\text{ham} \mid \text{email}) = \frac{P(\text{email} \mid \text{ham}) \cdot P(\text{ham})}{P(\text{email})} $$

$ P(\text{email}) > 0 $ olduğundan ve her iki ifadede de görüldüğünden, iki olasılığı karşılaştırmak için yalnızca payların değerlendirilmesi yeterlidir ve bu paydayı göz ardı edebilirsiniz.

<a name="2"></a>
## 2 - Gerekli içe aktarımlar

Bu sonraki kod bloğu, ödevde ihtiyaç duyacağınız tüm gerekli kütüphaneleri ve işlevleri ve çalışırken geri bildirim sağlayacak birim testlerini içe aktaracaktır.

In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
import string



In [3]:
import w1_unittest

<a name="3"></a>
## 3 - Veri Kümesi

<a name="3.1"></a>
### 3.1 Veri Kümesini Yükleme ve İnceleme

Aşağıdaki kod bloğu veri kümesini belleğe yükleyecektir. Pandas [DataFrame](https://pandas.pydata.org/docs/index.html) olarak okumak için [Pandas Kütüphanesini](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame) kullanacaksınız, birincil nesnesi. Ancak, Pandas'a hala aşinalık kazanıyorsanız **endişelenmenize gerek yok**. Veri yapısını göstermek için yüklenecek ve gerçek işiniz için NumPy dizileriyle sonuçlanacaksınız.

In [4]:
dataframe_emails = pd.read_csv('emails.csv')
dataframe_emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


Veri setini biraz inceleyelim:

In [5]:
print(f"E-posta sayısı: {len(dataframe_emails)}")
print(f"Spam e-postaların oranı: {dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")
print(f"ham e-postaların oranı: {1-dataframe_emails.spam.sum()/len(dataframe_emails):.4f}")

E-posta sayısı: 5728
Spam e-postaların oranı: 0.2388
ham e-postaların oranı: 0.7612


Bu veri kümesinin **dengesiz** olduğunu unutmayın. İçinde spam e-postalarından üç kat daha fazla ham e-posta var! Bu, herhangi bir veri analizi projesinde bilinmesi gereken yararlı bir bağlamdır ve Naive Bayes dahil olmak üzere bazı makine öğrenimi algoritmalarının nasıl çalıştığını etkileyebilir.

<a name="3.2"></a>
### 3.2 Veri setinin ön işlenmesi

DataFrame'in iki sütunu vardır. `text` adlı sütun e-postanın içeriğini, `spam` adlı sütun ise e-postanın spam olup olmadığını belirten sayısal bir değişkeni içerir. $1$'in spam, $0$'ın ise ham (spam değil) anlamına geldiğini unutmayın. Bir sonraki fonksiyon birkaç önemli ön işleme adımını tamamlayacaktır:

* Her e-postanın `Subject:` ile başladığını unutmayın. Bu fonksiyon bu kelimeyi her e-postanın önünden kaldıracaktır.
* Veri setini rastgele karıştıracaktır. Şu anda tüm spam e-postalar veri setinin en üstünde, ardından ham e-postalar geliyor. Verileri eğitim ve test veri setleri arasında düzgün bir şekilde bölmek için karıştırılmış bir veri setine ihtiyacınız var.

Bu fonksiyondaki tüm Python'u anlamasanız bile endişelenmeyin, ancak genellikle analize başlamadan önce verilerinizi keşfetmeniz ve ön işleme tabi tutmanız gerektiğini hatırlatmak için buraya eklenmiştir.

In [6]:
def preprocess_emails(df):
    """
Bir DataFrame'den e-posta verilerini önceden işler.

Parametreler:
- df (pandas.DataFrame): 'text' ve 'spam' sütunlarına sahip e-posta verilerini içeren giriş DataFrame'i.

Döndürür:
- tuple: İki öğe içeren bir tuple:
   1. X (numpy.array): "Konu:" öneki kaldırıldıktan sonra e-posta içeriğini içeren bir dizi.
   2. Y (numpy.array): Her e-postanın spam (1) veya ham (0) olup olmadığını gösteren bir dizi.

İşlev, eğitim/test bölmelerinde önyargılı sonuçları önlemek için giriş DataFrame'ini karıştırır.
Daha sonra e-posta içeriğini ve spam etiketlerini çıkarır ve her e-postadan "Konu:" önekini kaldırır.

"""
    # Veri setini karıştırır
    df = df.sample(frac = 1, ignore_index = True, random_state = 42)
    # Her e-postanın ilk 9 karakterini içeren "Konu:" dizesini kaldırır. Ayrıca, bunu bir numpy dizisine dönüştürün.
    X = df.text.apply(lambda x: x[9:]).to_numpy()
    # Etiketleri numpy dizisine dönüştür
    Y = df.spam.to_numpy()
    return X, Y

In [7]:
X, Y = preprocess_emails(dataframe_emails)

İlk $5$ e-postayı yazdıralım:

In [8]:
print(X[:5])

['re : energy derivatives conference - may 29 , toronto  good morning amy :  vince kaminski will need the following :  an lcd projector to hook up to a lap tap for his presentation  he will have dinner with the conference organizers and speakers on the 29 th .  he will need 2 nights ( the 28 th and the 29 th ) hotel reservations .  he will send you an abstract shortly .  thanks and have a great day !  shirley crenshaw  713 - 853 - 5290  amy aldous on 03 / 31 / 2000 10 : 50 : 11 am  to : shirley . crenshaw @ enron . com  cc :  subject : re : energy derivatives conference - may 29 , toronto  ms . crenshaw ,  thank you for sending the bio so quickly . it \' s exactly what i was looking  for .  we are planning to compile the conference speakers \' papers for distribution  to the participants . while i will not need dr . kaminski \' s contribution for  several weeks , an abstract of his presentation as soon as possible would be  very useful to the conference organizers .  i will also need t

Ve ilk $5$ etiket:

In [9]:
print(Y[:5])

[0 0 0 0 0]


Numpy dizisi `X`'in bir dize dizisi olduğunu unutmayın, bu nedenle bu dizideki her öğe bir e-postadır ve bu dizideki aynı dizin, e-postanın spam olup olmadığını söyleyen `Y`'deki dizindir. Çeşitli e-postaların metnini ve spam olup olmadıklarını görmek için **`email_index`**'teki değeri değiştirmeyi deneyin. 0'ın ham, 1'in ise spam anlamına geldiğini unutmayın.

In [10]:
email_index = 30
print(f"E-posta dizini {email_index}: {X[email_index]}\n\n")
print(f"Sınıf: {Y[email_index]}")

E-posta dizini 30: karthik rajan - interview schedule  attached you will find the interview packet for the above - referenced person .  the interview will happen friday , march 30 , 2001 . please print all three  documents for your hard copies . if you have any questions , or conflicts of  schedule , please do not hesitate to contact me .  sasha divelbiss  58714


Sınıf: 0


<a name="3.3"></a>
### 3.3 Metnin ön işlenmesi

Bu bölüm derslerde ele alınmamıştır ve içinde notlandırılmış bir işlev yoktur. Ancak, metinle uğraşırken önemlidir ve bunu öğrenmek Makine Öğrenimi ve Veri Bilimi yolculuğunuzda kesinlikle yardımcı olacaktır!

Metinde, genellikle metnin ne söylediği hakkında fazla bilgi sağlamayan bazı kelimeler vardır, örneğin edatlar, zamirler vb. Bunlara **durdurma sözcükleri** denir. Her metinde çok yaygın oldukları için, görevimiz için anlamlı bir bilgi depolamaları neredeyse imkansızdır. Amaç, tüm bu durdurma sözcüklerini ve noktalama işaretlerini kaldırmaktır, böylece sonunda başa çıkmanız gereken daha basit bir kelime kümesine sahip olursunuz. Bir sonraki işlevin yapacağı şey budur.

Bir diğer adım e-postaların **tokenizasyonu**'dur. Tokenizasyon, e-postayı **tokenlara** bölmektir, bunlar esasen içindeki kelimelerdir. Sonuç olarak her e-posta için nihai sonuç, durdurma sözcükleri ve noktalama işaretleri olmaksızın e-postadaki tüm kelimelerden oluşan bir numpy dizisi olacaktır.

In [11]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
def preprocess_text(X):
    """
    Durdurma sözcüklerini ve noktalama işaretlerini kaldırarak bir metin verisi koleksiyonunu önceden işler.
 
    Parametreler:
    - X (str veya dizi benzeri): İşlenecek giriş metin verisi. Tek bir dize sağlanırsa,
    tek öğeli bir numpy dizisine dönüştürülür.

    Döndürür:
    - numpy.array: Her bir öğenin durdurma sözcükleri ve noktalama işaretleri kaldırılmış bir belgeyi temsil ettiği, önceden işlenmiş metin verisi dizisi.
 
    Not:
    - İşlev, belirteçleme ve durdurma sözcüğü kaldırma için Doğal Dil Araç Takımı (nltk) kitaplığını kullanır.
    - Giriş tek bir dizeyse, tek öğeli bir numpy dizisine dönüştürülür.
    """
    # Durdurma sözcükleri ve noktalama işaretleriyle bir set oluşturun
    stop = set(stopwords.words('english') + list(string.punctuation))

    # Bir sonraki satırlar, bir dizi e-posta yerine tek bir e-postanın iletildiği durumu ele alacaktır.
    if isinstance(X, str):
        X = np.array([X])

    # Sonuç bir listede saklanacaktır
    X_preprocessed = []

    for email in X:
        words = re.findall(r'\b\w+\b', email.lower())  # Noktalama işaretlerini temizle
        filtered_words = [word for word in words if word not in ENGLISH_STOP_WORDS]
        X_preprocessed.append(filtered_words)
    
    return X_preprocessed if len(X) > 1 else X_preprocessed[0]

        
        

In [12]:
X_treated = preprocess_text(X)

Ön işlemeden sonra, her e-postanın metni tüm durdurma sözcükleri kaldırılmış bir numpy dizisine dönüştürüldü. Buradaki örnek, rastgele seçilen bir `email_index` değerinin (bu durumda 989) bu işleme adımından önce ve sonra nasıl göründüğünü gösterir. Bu adımın farklı e-postalardaki sonuçlarını görmek için farklı değerleri denemekten çekinmeyin. Her e-posta için bu temizlenmiş sözcük dizisi, algoritma tarafından gerçekte kullanılan şey olacaktır.

In [13]:
email_index = 989
print(f"Ön işlemeden önce e-posta: {X[email_index]}")
print(f"Ön işlemeden sonra e-posta: {X_treated[email_index]}")

Ön işlemeden önce e-posta: marketing for your espeak session  vince :  thanks for your time earlier this week ; i ' m looking forward to your espeak  event .  sarah and i met with our etv contact yesterday , and we will be able to put a  bulleted list on the elevator screens to advertise your espeak . please let  me know what you would like us to post for you , and we will do the rest !  we also have plans to market specifically to the trader community here at  enron , so you should get a high participation rate , especially from those  groups .  thanks , again .  - er
Ön işlemeden sonra e-posta: ['marketing', 'espeak', 'session', 'vince', 'thanks', 'time', 'earlier', 'week', 'm', 'looking', 'forward', 'espeak', 'event', 'sarah', 'met', 'etv', 'contact', 'yesterday', 'able', 'bulleted', 'list', 'elevator', 'screens', 'advertise', 'espeak', 'let', 'know', 'like', 'post', 'rest', 'plans', 'market', 'specifically', 'trader', 'community', 'enron', 'high', 'participation', 'rate', 'especial

<a name="3.4"></a>
### 3.4 Eğitim/test olarak ayırma

Şimdi veri setimizi eğitim ve test setlerine ayıralım. %80/20 oranında çalışacaksınız, yani verilerin %80'i eğitim için, %20'si ise test için kullanılacak.

In [14]:
TRAIN_SIZE = int(0.80*len(X_treated)) # Örneklerin %80’i eğitim için kullanılacak.

X_train = X_treated[:TRAIN_SIZE]
Y_train = Y[:TRAIN_SIZE]
X_test = X_treated[TRAIN_SIZE:]
Y_test = Y[TRAIN_SIZE:]

E-postaların yaklaşık %24'ünün spam olduğunu unutmayın. Bu oranın eğitim ve test veri kümelerinde aşağı yukarı aynı kalıp kalmadığını kontrol etmek önemlidir, aksi takdirde önyargılı bir algoritma oluşturabilirsiniz.

In [15]:
print(f"eğitim veri kümesindeki spam oranı: {sum(Y_train == 1)/len(Y_train):.4f}")
print(f"test veri kümesindeki spam oranı: {sum(Y_test == 1)/len(Y_test):.4f}")

eğitim veri kümesindeki spam oranı: 0.2431
test veri kümesindeki spam oranı: 0.2216


Eşit değiller ama çok yakınlar, o yüzden sorun yok!

<a name="4"></a>
## 4 - Naive Bayes Algoritmasını Uygulama

Görevinizi hatırlayın: Hangisinin daha büyük olduğuna karar vermek için $P(\text{spam} \mid \text{email})$ ve $P(\text{ham} \mid \text{email})$ değerlerini karşılaştırın. Karşılaştırmayı yapmak için yalnızca $P(\text{spam}) \cdot P(\text{email} \mid \text{spam})$ ve $P(\text{ham}) \cdot P(\text{email} \mid \text{ham})$ değerlerini hesaplamak yeterlidir.

<a name="4.1"></a>
### 4.1 $P(\text{email} \mid \text{spam})$ ve $P(\text{email} \mid \text{ham})$ değerlerini hesaplama

Her iki durum da aynı şekilde çalışır, bu yüzden spam durumuna başlayalım.

Her e-posta bir kelime listesidir. Amacınız, e-postanın spam olduğu varsayıldığında, bu kelime listesini görme olasılığınızı hesaplamaktır. Bunu yapmanın yolu, ürün kuralını uygulamaktır. Bir e-postayı $\text{email} = \{\text{word}_1, \text{word}_2, \ldots, \text{word}_n \}$ olarak temsil ederek, hesaplama şu şekildedir:

$$P(\text{email} \mid \text{spam}) = P(\text{word}_1 \mid \text{spam}) \cdot P(\text{word}_2 \mid \text{spam}) \cdots P(\text{word}_n \mid \text{spam})$$

Burada, "Naive Bayes" ismine yol açan **naif varsayımı** yaparsınız! Her bir kelimenin bir e-postada görünme olasılığının diğer her bir kelimenin olasılığından bağımsız olduğunu varsayarsınız. Bu varsayım elbette yanlıştır. "Party" kelimesini içeren e-postaların "invitation" kelimesini içerme olasılığı daha yüksektir. "Prize" kelimesini içeren e-postaların "tebrikler" kelimesini içerme olasılığı daha yüksektir. Ancak bu olasılıkların bağımsız olduğu yönünde yanlış bir varsayımda bulunarak, çarpım kuralını uygulama yeteneği kazanırsınız. Kelimeler arasındaki karmaşık bir koşullu olasılık kümesini hesaba katmak yerine, basitçe bağımsızlığı varsayabilir ve yukarıdaki ifadede gösterildiği gibi oldukça basit bir koşullu olasılık kümesini çarpabilirsiniz. Naive Bayes, verileriniz hakkında yanlış bir varsayım üzerine kuruludur, ancak göreceğiniz gibi, genellikle etkileyici sonuçlar verir!

İşte spam olduğu göz önüne alındığında, bir e-postada $\text{word}_1$ görünme olasılığını hesaplamanın yolu:

$$P(\text{word}_1 \mid \text{spam}) = \frac{\text{\# spam e-postalar } \text{word}_1}{\text{\# spam e-postalar}}$$

Burada \# sembolü öğe sayısını ifade eder, yani $\text{\# spam e-postalar } \text{word}_1$ $\text{word}_1$ içeren spam e-postaların miktarını ifade eder.

Bu aslında çok basit bir hesaplamadır. Kaç tane spam e-postanın $\text{word}_1$ içerdiğini sayın ve toplam spam e-posta sayısına bölün. Veri kümesindeki her kelimeyi yineleyin ve işlemi tekrarlayın, böylece spam veya amatör olduğu göz önüne alındığında, belirli bir e-postayı görmenin genel olasılığını hesaplamaya hazırsınız. Bunu akılda tutarak, **ilk göreviniz, veri kümesindeki her kelimenin ham ve spam e-postalarında görünme sıklığını depolamak için `word_frequency` adlı bir sözlük oluşturmak olacaktır**

#### 4.1.1 Üründe 0'ı İşleme

Yalnızca spam e-postalarında görünen veya hiçbir zaman spam e-postasında görünmeyen bir kelimeyle karşılaşmak, $P(\text{word} \mid \text{spam}) = 0$ (veya ham benzeri) ile sonuçlanabilir ve tüm ürünün $0$ olmasına yol açabilir. Bu senaryo istenmeyen bir durumdur çünkü tek bir kelime tüm olasılığı $0$ yapabilir. Bunu hafifletmek için, **1'den itibaren her kelime için spam/ham görünümlerini sayarak başlayacaksınız**. Her kelimeyle birlikte en az bir spam ve bir ham e-postası olduğunu yapay olarak varsayarak, hesaplamalarda $0$ görünme olasılığını ortadan kaldırırsınız.

<a name="4.2"></a>
### 4.2 $P(\text{spam})$ ve $P(\text{ham})$'ı hesaplama

Bayes Teoremi'ni kullanırken, ham ve spam e-postaları görmenin genel olasılığını da eklemeniz gerekir. Bu hesaplama oldukça kolaydır çünkü bunlar yalnızca veri kümesindeki spam ve ham e-postalarının oranıdır.

$$P(\text{spam}) = \frac{\text{\# spam e-postaları}}{\text{\# toplam e-postalar}}$$
$$P(\text{ham}) = \frac{\text{\# ham e-postaları}}{\text{\# toplam e-postalar}}$$

<a name="4.3"></a>
### 4.3 Hepsini bir araya getirme

Bir e-postanın spam veya ham olma olasılığını hesaplamak için, daha önce hesapladığınız terimleri çarpmanız ve hangisinin daha büyük olduğunu karşılaştırmanız yeterlidir.

- $P(\text{spam}) \cdot P(\text{e-posta} \mid \text{spam})$
- $P(\text{ham}) \cdot P(\text{e-posta} \mid \text{ham})$

<a name="ex01"></a>
### Alıştırma 1

Göreviniz, veri kümesindeki her bir kelimenin spam (1) veya ham (0) olarak görünme sıklığını kaydeden bir sözlük üreten işlevi uygulamaktır.

In [16]:
def get_word_frequency(X, Y):
    """
    Spam (1) veya spam değil (0) olarak kategorilendirilen bir e-posta kümesindeki her bir kelimenin sıklığını hesaplayın.
    
    Parametreler:
    - X (numpy.array): Her e-postanın bir kelime listesi olarak temsil edildiği e-posta dizisi.
    - Y (numpy.array): X'teki her e-postaya karşılık gelen etiket dizisi. 1 spam'i, 0 ise jambon'u belirtir.
    
    Dönüşler:
    - word_dict (sözlük): Anahtarların e-postalarda bulunan benzersiz kelimeler olduğu ve değerlerin spam (1) ve spam değil (0) e-postalar için her bir kelimenin sıklığını içeren sözlükler olduğu bir sözlük.
    """
    # Boş bir sözlük oluşturun
    word_dict = {}
        
    # Tüm benzersiz kelimeleri topla
    all_words = set()
    for email in X:
        for word in email:
            all_words.add(word)
    
    # Her kelime için başlangıç değerleri oluştur
    # 4.1.1'e göre, her kelime için ham ve spam sayımlarını 1'den başlatıyoruz
    for word in all_words:
        word_dict[word] = {"spam": 1, "ham": 1}
    
    # E-posta ve sınıflarına göre kelime sayımlarını güncelle
    for i in range(len(X)):
        email = X[i]
        cls = Y[i]
        
        # Her kelimeyi sadece bir kez say (e-postada geçiyorsa)
        unique_words = set(email)
        for word in unique_words:
            if cls == 1:  # Spam
                word_dict[word]["spam"] += 1
            else:  # Ham
                word_dict[word]["ham"] += 1
    
    return word_dict

In [17]:
test_output = get_word_frequency([['like','going','river'], ['love', 'deep', 'river'], ['hate','river']], [1,0,0])
print(test_output)

{'like': {'spam': 2, 'ham': 1}, 'river': {'spam': 2, 'ham': 3}, 'hate': {'spam': 1, 'ham': 2}, 'deep': {'spam': 1, 'ham': 2}, 'love': {'spam': 1, 'ham': 2}, 'going': {'spam': 2, 'ham': 1}}


##### __Beklenen Çıktı__ (çıktı sırası değişebilir, önemli olan her kelime için değerlerdir)

```Python
{'going': {'spam': 2, 'ham': 1}, 'river': {'spam': 2, 'ham': 3}, 'like': {'spam': 2, 'ham': 1}, 'deep': {'spam': 1, 'ham': 2}, 'love': {'spam': 1, 'ham': 2}, 'hate': {'spam': 1, 'ham': 2}}
```

Bir sonraki kod bloğu fonksiyonunuzu test edecektir. Bu sadece fonksiyonunuzun düzgün çalıştığından emin olmanızı sağlayacaktır. Birim testi başarısız olursa, bir sonraki egzersize geçmeden önce fonksiyonunuzu gözden geçirebilmeniz için geri bildirim alacaksınız.

In [25]:
w1_unittest.test_get_word_frequency(get_word_frequency)

 All tests passed


In [18]:
word_frequency = get_word_frequency(X_train,Y_train)

Ayrıca bir sınıf sıklığı sözlüğüne ihtiyacınız olacak. Bu, veri kümesindeki toplam ham (0) ve spam (1) e-posta sayısını depolayacaktır. Aşağıdaki kod satırı bunu sizin için oluşturacaktır.

In [19]:
# Spam ve ham e-postalarını saymak için, eğitim veri setindeki ilgili 1 ve 0 değerlerini toplayabilirsiniz; çünkü kural spam = 1 ve ham = 0'dır.
class_frequency = {'ham': sum(Y_train == 0), 'spam': sum(Y_train == 1)}

In [20]:
print(class_frequency)

{'ham': np.int64(3468), 'spam': np.int64(1114)}


Eğitim veri setindeki spam oranını almak için şunları yapabilirsiniz:

In [21]:
# Fikir, (spam e-postalarının miktarı)/(toplam e-postalar) hesaplamaktır.
# Bir e-posta spam veya ham olduğundan, toplam e-postalar = (ham e-postalarının miktarı) + (spam e-postalarının miktarı).
proportion_spam = class_frequency['spam']/(class_frequency['ham'] + class_frequency['spam'])
print(f"Eğitimde spam e-postaların oranı: {proportion_spam:.4f}")

Eğitimde spam e-postaların oranı: 0.2431


Dikkat edin, bu aşağıdaki hücrelerde elde ettiğiniz değerle uyuşuyor!

<a name="ex02"></a>
### Alıştırma 2

Bir sonraki alıştırmada, $P(\text{word} \mid \text{spam})$ ve $P(\text{word} \mid \text{ham})$'ı hesaplamak için fonksiyonu uygulayacaksınız. Her iki e-posta türü için de hesaplamalar aynı olduğundan, sınıfın spam ($1$) veya (ham) $0$ olabileceği $P(\text{word} \mid \text{class})$'ı hesaplamak için bir fonksiyon oluşturacaksınız.

Şunu unutmayın

$$P(\text{word}_i \mid \text{class}) = \frac{\text{\# sınıftaki e-postalar (spam veya ham) } \text{word}_i}{\text{\# verilen sınıftaki e-postalar (spam veya ham)}}$$

**Şimdilik bir kelimenin sözlükte olup olmadığı konusunda endişelenmeyeceğinizi unutmayın. Bu, daha sonraki fonksiyonlarda ele alınacaktır.**

In [22]:


def prob_word_given_class(word, cls, word_frequency, class_frequency):
    """
    Belirli bir kelimenin belirli bir sınıfta (spam veya ham) ortaya çıkma olasılığını hesaplar.

    Parametreler:
    - word (str): Olasılığı hesaplanacak kelime.
    - cls (str): "spam" veya "ham" sınıfı.
    - word_frequency (dict): Kelimelerin belirli sınıflarda kaç kez geçtiğini içeren sözlük.
    - class_frequency (dict): Her sınıftaki toplam e-posta sayısını içeren sözlük.

    Dönüş:
    - float: Kelimenin belirli bir sınıfta geçme olasılığı.
    """
    amount_word_and_class = word_frequency.get(word, {}).get(cls, 0)  # Kelime ve sınıf var mı diye kontrol et
    total_class_emails = class_frequency.get(cls, 1)  # Sınıf sayısını al, yoksa 1 döndür
    
    return amount_word_and_class / total_class_emails

In [23]:
print(f"P(lottery | spam) = {prob_word_given_class('lottery', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(lottery | ham) = {prob_word_given_class('lottery', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | spam) = {prob_word_given_class('schedule', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | ham) = {prob_word_given_class('schedule', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")

P(lottery | spam) = 0.00807899461400359
P(lottery | ham) = 0.0002883506343713956
P(schedule | spam) = 0.008976660682226212
P(schedule | ham) = 0.10294117647058823


##### __Beklenen Çıktı__ (sonuçlar son ondalık basamaklarda değişiklik gösterebilir)

```Python
P(piyango | spam) = 0.00807899461400359
P(piyango | ham) = 0.0002883506343713956
P(zamanlama | spam) = 0.008976660682226212
P(zamanlama | ham) = 0.10294117647058823
```

<a name="ex03"></a>
### Alıştırma 3

Bir sonraki alıştırmada, sınıfın spam (1) veya ham (0) olabileceği $P(\text{email} \mid \text{class})$ değerini hesaplamak için işlevi uygulayacaksınız. Şu *saf varsayımı* kullanacaksınız:

$$P(\text{email} \mid \text{class}) = P(\text{word}_1 \mid \text{class}) \cdot P(\text{word}_2 \mid \text{class}) \cdots P(\text{word}_n \mid \text{class})$$

Fikir, e-postadaki her kelime üzerinde yineleme yapmak ve her adımda, $P(\text{word} \mid \text{class})$ değeriyle çarparak olasılığı güncellemektir.

Python'da değerleri güncellemek için `value = value * update` kullanmak yerine, sadece `value *= update` kullanabileceğinizi unutmayın. Aynı hesaplamayı yapıyorlar.

In [28]:
def prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    """
      İşlenmiş e-posta içeriğine göre bir e-postanın belirli bir sınıftan (örneğin spam veya ham) olma olasılığını hesaplayın.

      Parametreler:
      - treated_email (liste): E-postadaki işlenmiş kelimelerin listesi.
      - cls (str): E-postanın sınıf etiketi. 'spam' veya 'ham' olabilir.
      - word_frequency (sözlük): Frekans kelimelerini içeren sözlük.
      - class_frequency (sözlük): Frekans sınıfını içeren sözlük.

      Döndürür:
      - float: Belirtilen e-postanın belirtilen sınıfa ait olma olasılığı.
    """
    # prob 1'den başlar çünkü her yinelemede geçerli P(kelime | sınıf) ile çarpılarak güncellenecektir
    prob = 1

   
    for word in treated_email:
        # Hesaplamayı yalnızca kelime sıklığı sözlüğünde bulunan kelimeler için gerçekleştirin
        if word in word_frequency.keys():
            # Olasılığı P(kelime | sınıf) ile çarparak güncelle
            prob *= prob_word_given_class(word, cls, word_frequency, class_frequency)
    

    return prob


In [29]:
example_email = "Click here to win a lottery ticket and claim your prize!"
treated_email = preprocess_text(example_email)
prob_spam = prob_email_given_class(treated_email, cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)
prob_ham = prob_email_given_class(treated_email, cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)
print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nP(email | spam) = {prob_spam}\nP(email | ham) = {prob_ham}")

Email: Click here to win a lottery ticket and claim your prize!
Email after preprocessing: ['click', 'win', 'lottery', 'ticket', 'claim', 'prize']
P(email | spam) = 5.3884806600117164e-11
P(email | ham) = 1.2428344868918976e-15


##### __Beklenen Çıktı__ (sonuçlar son ondalık basamaklarda değişiklik gösterebilir)

```Python
E-posta: Bir piyango bileti kazanmak ve ödülünüzü talep etmek için buraya tıklayın!
Ön işlemeden sonra e-posta: ['click' 'win' 'lottery' 'ticket' 'claim' 'prize']
P(email | spam) = 5.3884806600117164e-11
P(email | ham) = 1.2428344868918976e-15
```

<a name="ex04"></a>
### Alıştırma 4

Bu alıştırmada, bir e-postanın spam veya ham olma olasılığını hesaplamak için aşağıdaki iki hesaplamayı gerçekleştireceksiniz:

- $ P(\text{spam}) \cdot P(\text{email} \mid \text{spam}) $

- $ P(\text{ham}) \cdot P(\text{email} \mid \text{ham})$

En büyük değere sahip olan, algoritmanızın o e-postaya atadığı sınıf olacaktır. Aşağıdaki işlevin, seçilen sınıf yerine işleve her iki olasılığı da döndürmesini söyleyen bir parametre içerdiğini unutmayın.

**Not**: Çıktının, ilgili e-posta sınıfını belirten bir tam sayı olacağını fark edeceksiniz. E-postanın spam olarak tahmin edilmesi durumunda spam, e-postanın ham olarak tahmin edilmesi durumunda ise ham döndürülmesi mümkün olabilir, ancak modelin bir sayı çıktısı vermesi, model performansını değerlendirmek için metrikler gibi daha fazla hesaplamaya yardımcı olur.

In [31]:
def naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood=False):
    """
    Naive Bayes sınıflandırıcısı ile spam tespiti.

    Bu fonksiyon, bir e-postanın spam (1) veya ham (0) olma olasılığını
    Naive Bayes algoritmasına dayalı olarak hesaplar. Bu, e-postanın spam ve ham olduğuna dair
    koşullu olasılıkları ve spam ve ham sınıflarının önsel olasılıklarını kullanır.
    Son karar, hesaplanan olasılıkları karşılaştırarak verilir.

    Parametreler:
    - treated_email (list): Girdi e-postasının önceden işlenmiş hali.
    - word_frequency (dict): Kelimelerin frekansını içeren sözlük.
    - class_frequency (dict): Sınıf frekanslarını içeren sözlük.
        - return_likelihood (bool): Eğer True ise, her iki sınıfın olasılığını döndürür.

    Döndürülen Değerler:
    Eğer return_likelihood = False ise:
        - int: E-posta spam olarak sınıflandırılmışsa 1, ham olarak sınıflandırılmışsa 0 döner.
    Eğer return_likelihood = True ise:
        - tuple: (spam_likelihood, ham_likelihood) formatında bir tuple döner.
    """

    # P(email | spam) değerini yukarıda tanımladığınız fonksiyonla hesaplayın.
    prob_email_given_spam = prob_email_given_class(treated_email, cls='spam', word_frequency=word_frequency, class_frequency=class_frequency)

    # P(email | ham) değerini yukarıda tanımladığınız fonksiyonla hesaplayın.
    prob_email_given_ham = prob_email_given_class(treated_email, cls='ham', word_frequency=word_frequency, class_frequency=class_frequency)

    # P(spam) değerini class_frequency sözlüğünü kullanarak hesaplayın: #spam e-posta / #toplam e-posta
    p_spam = class_frequency['spam'] / sum(class_frequency.values())

    # P(ham) değerini class_frequency sözlüğünü kullanarak hesaplayın: #ham e-posta / #toplam e-posta
    p_ham = class_frequency['ham'] / sum(class_frequency.values())

    # P(spam) * P(email | spam) hesaplayın, buna spam_likelihood diyelim
    spam_likelihood = p_spam * prob_email_given_spam

    # P(ham) * P(email | ham) hesaplayın, buna ham_likelihood diyelim
    ham_likelihood = p_ham * prob_email_given_ham

    # return_likelihood = True ise, istenen tuple'ı döndürün
    if return_likelihood == True:
        return (spam_likelihood, ham_likelihood)
    
    # Her iki değeri karşılaştırın ve daha yüksek olana sahip olan sınıfı seçin
    elif spam_likelihood >= ham_likelihood:
        return 1  # spam
    else:
        return 0  # ham


In [32]:
example_email = "Click here to win a lottery ticket and claim your prize!"
treated_email = preprocess_text(example_email)

print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")

print("\n\n")
example_email = "Our meeting will happen in the main office. Please be there in time."
treated_email = preprocess_text(example_email)

print(f"Email: {example_email}\nEmail after preprocessing: {treated_email}\nNaive Bayes predicts this email as: {naive_bayes(treated_email, word_frequency, class_frequency)}")

Email: Click here to win a lottery ticket and claim your prize!
Email after preprocessing: ['click', 'win', 'lottery', 'ticket', 'claim', 'prize']
Naive Bayes predicts this email as: 1



Email: Our meeting will happen in the main office. Please be there in time.
Email after preprocessing: ['meeting', 'happen', 'main', 'office', 'time']
Naive Bayes predicts this email as: 0


#### __Beklenen Çıktı__

```
E-posta: Bir piyango bileti kazanmak ve ödülünüzü talep etmek için buraya tıklayın!
Ön işlemeden sonra e-posta: ['tıkla' 'kazan' 'piyango' 'bilet' 'talep et' 'ödül']
Naive Bayes bu e-postayı şu şekilde tahmin ediyor: 1

E-posta: Toplantımız ana ofiste gerçekleşecek. Lütfen zamanında orada olun.
Ön işlemeden sonra e-posta: ['toplantı' 'gerçekleşecek' 'ana' 'ofis' 'lütfen' 'zaman']
Naive Bayes bu e-postayı şu şekilde tahmin ediyor: 0
```

Bir sonraki kod bloğu fonksiyonunuzu test edecektir. Bu sadece fonksiyonunuzun düzgün çalıştığından emin olmanızı sağlayacaktır. Birim testi başarısız olursa, bir sonraki egzersize geçmeden önce fonksiyonunuzu gözden geçirebilmeniz için geri bildirim alacaksınız.

In [33]:
w1_unittest.test_naive_bayes(naive_bayes, word_frequency, class_frequency)

 All tests passed


<a name="4.4"></a>
### 4.4 Model performansı

Bu bölüm, derslerde gördüklerinizin ötesine geçtiği için notlandırılmış bir bölüm içermez. Ancak, model performansını ölçmek modeller oluştururken çok önemli olduğundan, bunu okumanızı ve ne yapıldığını anlamaya çalışmanızı öneririz.

Bu bölümde, az önce oluşturduğunuz modelin performansını keşfedeceksiniz. Modelinizi verilerin %80'i üzerinde eğittiğinizi ve test etmek için verilerinizin %20'sini rastgele test verisi olarak sakladığınızı hatırlayın. O zaman doğal soru, modelin test verilerinizde kullanıldığında ne sıklıkla doğru bir sınıflandırma yaptığıdır. Bu soruyu cevaplamak için [doğruluk](https://en.wikipedia.org/wiki/Accuracy_and_precision) adlı bir ölçüm vardır. Bu, modelin ne kadar doğru tahminde bulunduğunun bir ölçüsüdür.

Doğruluğu hesaplamak için şunları yapmalısınız:

- Modelin doğru bir şekilde spam olarak sınıflandırdığı her spam e-postayı sayın (bunlara **gerçek pozitifler** denir)
- Modelin doğru bir şekilde jambon olarak sınıflandırdığı her jambon e-postasını sayın (bunlara **gerçek negatifler** denir)

Son olarak, bir oran elde etmek için gerçek pozitiflerin ve gerçek negatiflerin toplamını toplam gözlem sayısına bölersiniz. Model mükemmelse, doğruluk 1 veya %100 olur. Bir sonraki kod bloğu bu hesaplamayı yapmak için işlevleri uygulayacaktır.

In [37]:
def get_true_positives(Y_true, Y_pred):
    """
    İkili sınıflandırmada gerçek pozitif örneklerin sayısını hesaplayın.

    Parametreler:
    - Y_true (liste): Her örnek için gerçek etiketlerin (0 veya 1) listesi.
    - Y_pred (liste): Her örnek için tahmin edilen etiketlerin (0 veya 1) listesi.
    
    Dönüşler:
    - int: Gerçek pozitiflerin sayısı, burada gerçek etiket ve tahmin edilen etiket her ikisi de 1'dir.
    """
    # Hem Y_true hem de Y_pred uzunluk olarak eşleşmelidir.
    if len(Y_true) != len(Y_pred):
        return "Gerçek etiket sayısı ile tahmini etiket sayısı mutlaka eşleşmelidir!"
    n = len(Y_true)
    true_positives = 0
    # Listedeki eleman sayısı üzerinde yineleme yap
    for i in range(n):
        # Dikkate alınan e-posta için gerçek etiketi alın
        true_label_i = Y_true[i]
        # Dikkate alınan e-posta için öngörülen (model çıktısını) alın
        predicted_label_i = Y_pred[i]
        # Sayacı yalnızca true_label_i = 1 ve predicted_label_i = 1 (doğru pozitifler) durumunda 1 artırın
        if true_label_i == 1 and predicted_label_i == 1:
            true_positives += 1
    return true_positives
        
def get_true_negatives(Y_true, Y_pred):
    """
    İkili sınıflandırmada gerçek negatif örneklerin sayısını hesaplayın.

    Parametreler:
    - Y_true (liste): Her örnek için gerçek etiketlerin (0 veya 1) listesi.
    - Y_pred (liste): Her örnek için tahmin edilen etiketlerin (0 veya 1) listesi.
    
    Dönüşler:
    - int: Gerçek negatiflerin sayısı, burada gerçek etiket ve tahmin edilen etiket her ikisi de 0'dır.
    """
    
    # Hem Y_true hem de Y_pred uzunluk olarak eşleşmelidir.
    if len(Y_true) != len(Y_pred):
        return "Gerçek etiket sayısı ile tahmini etiket sayısı mutlaka eşleşmelidir!"
    n = len(Y_true)
    true_negatives = 0
    # Listedeki eleman sayısı üzerinde yineleme yap
    for i in range(n):
        # Dikkate alınan e-posta için gerçek etiketi alın
        true_label_i = Y_true[i]
        # Dikkate alınan e-posta için öngörülen (model çıktısını) alın
        predicted_label_i = Y_pred[i]
        # Sayacı yalnızca true_label_i = 0 ve predicted_label_i = 0 (doğru negatifler) durumunda 1 artırın
        if true_label_i == 0 and predicted_label_i == 0:
            true_negatives += 1
    return true_negatives
        

In [38]:
# Test seti için tahminleri alalım:

# Tahminleri depolamak için boş bir liste oluştur
Y_pred = []


# Test kümesindeki her e-postayı yineleyin
for email in X_test:
    # Tahmini gerçekleştir
    prediction = naive_bayes(email, word_frequency, class_frequency)
    # Listeye ekle
    Y_pred.append(prediction)

# Y_pred ve Y_test'in (bunlar gerçek etiketlerdir) uzunluklarının eşleşip eşleşmediğini kontrol ediyoruz:
print(f"Y_test ve Y_pred eşleşmeleri uzunluk olarak eşitmi? Cevap: {len(Y_pred) == len(Y_test)}")

Y_test ve Y_pred eşleşmeleri uzunluk olarak eşitmi? Cevap: True


In [39]:
# Gerçek pozitiflerin sayısını al:
true_positives = get_true_positives(Y_test, Y_pred)

# Gerçek negatiflerin sayısını al:
true_negatives = get_true_negatives(Y_test, Y_pred)

print(f"Gerçek pozitiflerin sayısı: {true_positives}\nGerçek negatiflerin sayısı: {true_negatives}")

# Doğruluğu, gerçek negatifleri gerçek pozitiflerle toplayıp veri kümesindeki toplam eleman sayısına bölerek hesaplayın.
# Hem Y_pred hem de Y_test aynı uzunluğa sahip olduğundan hangisini kullandığınız önemli değildir.
accuracy = (true_positives + true_negatives)/len(Y_test)

print(f"Doğruluk: {accuracy:.4f}")

Gerçek pozitiflerin sayısı: 249
Gerçek negatiflerin sayısı: 738
Doğruluk: 0.8613


Harika iş! E-postadaki her kelimenin tek başına durduğunu varsayarak sağlam bir Naive Bayes modeli geliştirdiniz. Bu temel yaklaşımla bile, model etkileyici bir şekilde %86,13'lik bir doğruluğa ulaşıyor! Aferin! Şimdi, bir sonraki kod bloğunda, devam edin ve kendi e-postanızı oluşturun. Şimdi bir sonraki kod bloğunda modelinizle deneyler yapabilirsiniz.

In [40]:
email = "Please meet me in 2 hours in the main building. I have an important task for you."
# email = "Piyango ödülü kazandınız! Tebrikler! Talep etmek için buraya tıkla"

# E-postayı önceden işle
treated_email = preprocess_text(email)
# Tahmini al, güzel bir şekilde yazdırmak için, çıktı 1 ise tahmin "spam", aksi takdirde "ham" olarak yazılacak.
prediction = "spam" if naive_bayes(treated_email, word_frequency, class_frequency) == 1 else "ham"
print(f"The email is: {email}\nThe model predicts it as {prediction}.")

The email is: Please meet me in 2 hours in the main building. I have an important task for you.
The model predicts it as ham.


<a name="5"></a>
## 5 - Ek Bölüm 

Aşağıdaki bölümler derecelendirilmemiştir ancak az önce yaptığınız çalışmanın bazı ilginç uzantılarını göstermektedir. Ancak daha derinlemesine bilgi edinmek isterseniz aşağıdaki bölümlere göz atabilirsiniz.

<a name="5.1"></a>
### 5.1 Naive Bayes modelindeki gizli sorun.

Mevcut modeldeki gizli bir sorun performansını etkiliyor. Belirli bir örnekte Naive Bayes hesaplamasını manuel olarak gerçekleştirerek sorunu inceleyelim.

In [41]:
example_index = 4798
example_email = X[example_index]
treated_email = preprocess_text(example_email)
print(f"The email is:\n\t{example_email}\n\nAfter preprocessing:\n\t:{treated_email}")

The email is:
	from the enron india newsdesk - may 5 - 7 newsclips  stinson / vince ,  some news articles . do read the first one , and the second last one .  regards ,  sandeep .  - - - - - - - - - - - - - - - - - - - - - - forwarded by sandeep kohli / enron _ development on  05 / 07 / 2001 09 : 10 am - - - - - - - - - - - - - - - - - - - - - - - - - - -  nikita varma  05 / 07 / 2001 07 : 42 am  to : nikita varma / enron _ development @ enron _ development  cc : ( bcc : sandeep kohli / enron _ development )  subject : from the enron india newsdesk - may 5 - 7 newsclips  the economic times , may 7 , 2001  enron ceo casts vote to save dpc , tina edwin & soma banerjee  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  the economic times , may 7 , 2001  maha sore over delay in naming godbole nominee  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -  the times of india , 7 may , 2001  maharashtra ' unhappy ' with delay in naming godbole nominee  - - - 

Bu durumda $P(\text{spam}) \cdot P(\text{email} \mid \text{spam})$ ve $P(\text{ham}) \cdot P(\text{email} \mid \text{ham})$'ı hesaplayalım. Bunu `naive_bayes` fonksiyonuna `return_likelihood = True` argümanını geçirerek yapabilirsiniz.

In [42]:
spam_likelihood, ham_likelihood = naive_bayes(treated_email, word_frequency = word_frequency, class_frequency = class_frequency, return_likelihood = True)
print(f"spam_likelihood: {spam_likelihood}\nham_likelihood: {ham_likelihood}")

spam_likelihood: 0.0
ham_likelihood: 0.0


Bu garip, hem spam hem de ham olasılığı $0$! Nasıl mümkün olabilir? Bu arada, gerçek kurala göre, model $\text{spam\_likelihood} \geq \text{ham\_likelihood}$ ise 1 (spam) olarak sınıflandırır, bu yüzden bu e-posta spam olarak sınıflandırılır. Gerçek ve öngörülen etiketleri karşılaştıralım.

In [43]:
print(f"Örnek e-posta şu şekilde etiketlenmiştir: {Y[example_index]}")
print(f"Naive Bayes modeli bunu şu şekilde sınıflandırır: {naive_bayes(treated_email, word_frequency, class_frequency)}")

Örnek e-posta şu şekilde etiketlenmiştir: 0
Naive Bayes modeli bunu şu şekilde sınıflandırır: 1


Yani, bu spam klasörüne yanlışlıkla gönderilecek bir e-posta! Ancak, bu davranışın tuhaf olduğunu unutmayın çünkü her iki olasılık da $0$'dır. Bu nasıl mümkün olabilir? Cevap, bunun ardındaki matematikte yatıyor!

Naive Bayes için ana hesaplamayı düşünün:

$$P(\text{email} \mid \text{spam}) = P(\text{word}_1 \mid \text{spam}) \cdot P(\text{word}_2 \mid \text{spam}) \cdots P(\text{word}_n \mid \text{spam})$$

E-postadaki **her** kelimenin bir ürünüdür.

In [44]:
print(f"Örnek e-postada üründe {len(treated_email)} kelime var.")

Örnek e-postada üründe 2424 kelime var.


Yani incelediğiniz e-postada $2657$ kelime var! E-postadaki ilk 3 kelime için $P(\text{word} \mid \text{ham})$ değerini hesaplayalım:

In [45]:
for i in range(3):
    word = treated_email[i]
    p_word_given_ham = prob_word_given_class(word, cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)
    print(f"Word: {word}. P({word} | ham) = {p_word_given_ham}")

Word: enron. P(enron | ham) = 0.5957324106113033
Word: india. P(india | ham) = 0.01787773933102653
Word: newsdesk. P(newsdesk | ham) = 0.0017301038062283738


Hepsi olasılık olduğu için, bunlar $0$ ile $1$ arasındaki sayılardır. Dolayısıyla, gerçekleştirilen ürün $0$ ile $1$ arasındaki $2657$ sayının ürünüdür. Her kelimenin $10^{-1}$ büyüklüğünde bir olasılığa sahip olduğu en iyi senaryoda (yukarıdaki örnekteki ilk kelimeye benzer şekilde), ortaya çıkan olasılık $10^{-2657}$ büyüklüğünde olacaktır; bu, herhangi bir bilgisayarın hassasiyetle işlemesi için zor olan **çok küçük bir sayıdır**. Python'un kayan nokta sayıları (ondalık sayılar) üzerindeki sınırını inceleyelim:

In [46]:
import sys

print(sys.float_info)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


Gördüğünüz gibi, minimum float değeri $10^{-308}$ büyüklüğündedir, $10^{-2657}$'den önemli ölçüde daha büyüktür. Sonuç olarak, Python ürünün sonucunu bir noktada $0$ olarak yorumlar ve bu da tüm bilgilerin kaybolmasına yol açar. Başka bir deyişle, algoritmanızın şu anda yazıldığı şekilde, belirli bir uzunluğun ötesinde, tüm e-postalar spam olarak sınıflandırılır. Naive Bayes tarafından gerekli görülen çok büyük üründe kök salan bu sorunun doğası göz önüne alındığında, sorunu ele almak çok önemlidir.

#### 5.1.1 Alt Taşma Sorunu

Karşılaştığınız zorluk, **alt taşma sorunu** olarak adlandırılır ve bilgisayarın hassasiyetinin ötesinde aşırı küçük sayılarla uğraştığınızı gösterir. Bu durumda, temel neden Naive Bayes hesaplamalarında yer alan **çok büyük üründür**. Neyse ki, bu soruna bir çözüm var.

Naive Bayes'te olasılıkların belirli değerlerinin kritik olmadığını hatırlayın çünkü algoritma yalnızca **değerleri karşılaştırır**. Bu nedenle aşağıdaki denklemlerdeki paydalar göz ardı edilmiştir:

$$ P(\text{spam} \mid \text{email}) = \frac{P(\text{spam}) \cdot P(\text{email} \mid \text{spam})}{P(\text{email})} $$
$$ P(\text{ham} \mid \text{email}) = \frac{P(\text{ham}) \cdot P(\text{email} \mid \text{ham})}{P(\text{email}) } $$

Hedef, ikisi arasındaki daha büyük değeri belirlemek ve aynı pozitif paydayı paylaştıkları için yalnızca paylar önemlidir. Özellikle, bu iki ürünün gerçek değerleri:

$$P(\text{spam}) \cdot P(\text{email} \mid \text{spam})$$
$$P(\text{ham}) \cdot P(\text{email} \mid \text{ham})$$

hangisinin diğerinden daha büyük olduğunu söyleyebildiğiniz sürece önemsizdir.

Bu miktarlara uygulanabilen ve **sıralamayı koruyan** bir fonksiyon varsa, o zaman bu değerlerin çıktılarını böyle bir fonksiyonda karşılaştırmak, maksimum değere sahip sınıfı belirleyecektir (gerçek sayısal değer farklı olsa da).

Herhangi bir **kesin artan fonksiyon** bu özelliğe sahiptir: maksimum **noktayı** korur. Bu nedenle, fikir Naive Bayes algoritmasının karşılaştığı büyük ürünü ele almaya yardımcı olan **artan bir fonksiyon** bulmaktır. Bir tane düşünebiliyor musunuz? Evet, var: $\log$ fonksiyonu. Bildiğiniz gibi, $\log$ **ürünleri** **toplamlara** dönüştürebilir! $\log$ arttığı için maksimum noktayı korur. Bu nedenle, aşağıdaki miktarları karşılaştırabilirsiniz:

$$\log \left(P(\text{spam}) \cdot P(\text{email} \mid \text{spam}) \right)$$
$$\log \left(P (\text{ham}) \cdot P(\text{email} \mid \text{ham}) \right)$$

Ve bu yeni miktarlar arasından maksimum değeri seçin. Sınıfı spam veya ham olarak belirtmek:

$$\log \left(P(\text{class}) \cdot P(\text{email} \mid \text{class}) \right) = \log \left(P(\text{class}) \right) + \log \left( P(\text{email} \mid \text{class}) \right)$$

Ve

$$\log \left( P(\text{email} \mid \text{class}) \right) = \log \left(P(\text{word}_1 \mid \text{class}) \cdot P(\text{word}_2 \mid \text{class}) \cdots P(\text{word}_n \mid \text{class}) \right) = \log \left(P(\text{word}_1 \mid \text{class}) \right) + \log \left(P(\text{word}_2 \mid \text{class})\right) + \cdots + \log \left( P(\text{word}_n \mid \text{class}) \right) $$

Bu yaklaşımla, büyük bir ürünü büyük bir toplama, sayısal olarak önemli ölçüde daha kararlı bir işleme dönüştürdünüz. Şimdi, bu yeni teknikle fonksiyonlarımızı iyileştireceksiniz! İki fonksiyonu ayarlamanız gerekiyor:

- `prob_email_given_class` - olasılık kelime ürününü logaritmaların toplamı ile değiştirin
- `naive_bayes` - $P(\text{class}) \cdot P(\text{email} \mid \text{class})$ ürününü ilgili logaritma toplamıyla değiştirin.

Yeni fonksiyonlar `log_prob_email_given_class` ve `log_naive_bayes` olarak adlandırılacaktır.

In [58]:
def log_prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    """
    Bu fonksiyon, bir e-postanın belirli bir sınıfa (örneğin, spam veya ham) ait olma olasılığının logaritmasını hesaplar.
    Hesaplama, e-posta içeriği üzerinde yapılan işlem sonrasında elde edilen kelimelere dayanır.

    Parametreler:
    - treated_email (list): E-posta içeriğindeki işlenmiş kelimelerden oluşan bir liste.
    - cls (str): E-posta sınıfının etiketi ('spam' veya 'ham')
    
    Dönen Değer:
    - float: Verilen e-postanın belirtilen sınıfa ait olma logaritmik olasılığı.
    """

    # Başlangıçta prob 0 olarak ayarlanır çünkü her kelime için log(P(kelime | sınıf)) hesaplanarak buna ekleme yapılacak
    prob = 0

    for word in treated_email: 
        # Sadece word_frequency sözlüğünde mevcut olan kelimeler için hesaplama yapılacak
        if word in word_frequency.keys(): 
            # prob değeri her bir kelime için log(P(kelime | sınıf)) ile güncelleniyor
            prob += np.log(prob_word_given_class(word, cls, word_frequency, class_frequency))

    return prob


In [59]:
# Sadece bir kelimeden oluşan bir e-postayı ele alalım, bu yüzden P(kelime | sınıf) veya log(P(kelime | sınıf)) değerini hesaplamak için indirgenir.
one_word_email = ['schedule']
word = one_word_email[0]
prob_spam = prob_email_given_class(one_word_email, cls = 'spam',word_frequency = word_frequency, class_frequency = class_frequency)
log_prob_spam = log_prob_email_given_class(one_word_email, cls = 'spam',word_frequency = word_frequency, class_frequency = class_frequency)
print(f"For word {word}:\n\tP({word} | spam) = {prob_spam}\n\tlog(P({word} | spam)) = {log_prob_spam}")

For word schedule:
	P(schedule | spam) = 0.008976660682226212
	log(P(schedule | spam)) = -4.713127327493184


$\text{log}$'un küçük bir sayıyı iyi büyüklükte negatif bir sayıya dönüştürebildiğini unutmayın. Dahası, algoritma artık ürün yerine toplama işlemi gerçekleştiriyor.

Bir sonraki kod bloğu log_naive_bayes'i uygular.

In [60]:
def log_naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood = False):    
    """
    Spam tespiti için Naive Bayes sınıflandırıcısı, gerçek olasılıklar yerine logaritmik olasılıkları karşılaştırır.

    Bu fonksiyon, Naive Bayes algoritmasını kullanarak bir e-postanın spam (1) veya ham (0) olma logaritmik olasılığını hesaplar. 
    Bu işlem, işlenmiş e-posta içeriği ve spam ve ham için koşullu olasılıkların yanı sıra, spam ve ham sınıflarının önceden belirlenen olasılıklarını kullanır. 
    Son karar, hesaplanan logaritmik olasılıkların karşılaştırılması ile yapılır.

    Parametreler:
    - treated_email (list): İşlenmiş e-posta içeriği (kelimelerden oluşan liste).
    - return_likelihood (bool): Eğer True ise, spam ve ham için logaritmik olasılıkları döndürüyor.

    Dönen Değer:
    - int: Eğer e-posta spam olarak sınıflandırılmışsa 1, ham olarak sınıflandırılmışsa 0 döner.
    """
    
    # P(e-posta | spam) hesaplanıyor, burada log fonksiyonu kullanılıyor
    log_prob_email_given_spam = log_prob_email_given_class(treated_email, cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency) 

    # P(e-posta | ham) hesaplanıyor, yine aynı şekilde log fonksiyonu ile
    log_prob_email_given_ham = log_prob_email_given_class(treated_email, cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency) 

    # P(spam) hesaplanıyor, class_frequency sözlüğünden kullanılarak #spam e-posta / #toplam e-posta
    p_spam = class_frequency['spam'] / (class_frequency['ham'] + class_frequency['spam']) 

    # P(ham) hesaplanıyor, class_frequency sözlüğünden kullanılarak #ham e-posta / #toplam e-posta
    p_ham = class_frequency['ham'] / (class_frequency['ham'] + class_frequency['spam']) 

    # log(P(spam)) + log(P(e-posta | spam)) hesaplanıyor, buna log_spam_likelihood diyoruz
    log_spam_likelihood = np.log(p_spam) + log_prob_email_given_spam 

    # log(P(ham)) + log(P(e-posta | ham)) hesaplanıyor, buna log_ham_likelihood diyoruz
    log_ham_likelihood = np.log(p_ham) + log_prob_email_given_ham 

    # Eğer return_likelihood True ise, her iki logaritmik olasılığı döndürüyoruz
    if return_likelihood == True:
        return (log_spam_likelihood, log_ham_likelihood)
    
    # Her iki değeri karşılaştırıp, daha yüksek olanın sınıfını seçiyoruz.
    # Çünkü logaritma artan bir fonksiyon olduğundan, daha yüksek olan sınıf bu özelliği korur.
    if log_spam_likelihood >= log_ham_likelihood:
        return 1  # Spam
    else:
        return 0  # Ham


Bölümün başındaki örneği tekrar ziyaret ederek `log_spam_likelihood` ve `log_ham_likelihood` değerlerini hesaplayacaksınız.

In [61]:
log_spam_likelihood, log_ham_likelihood = log_naive_bayes(treated_email,word_frequency = word_frequency, class_frequency = class_frequency,return_likelihood = True)
print(f"log_spam_olasılığı: {log_spam_likelihood}\nlog_ham_olasılığı: {log_ham_likelihood}")

log_spam_olasılığı: -10818.78118985717
log_ham_olasılığı: -9684.31898954805


Harika! Şimdi iki tane farklı sıfır olmayan sayı var! Daha yüksek olanın `log_ham_likelihood` olduğunu unutmayın, bu nedenle `log_naive_bayes` fonksiyonu bu e-postayı doğru bir şekilde ham olarak tahmin edecektir:

In [62]:
print(f"Örnek e-posta şu şekilde etiketlenmiştir: {Y[example_index]}")
print(f"Log Naive Bayes modeli bunu şu şekilde sınıflandırır: {log_naive_bayes(treated_email,word_frequency = word_frequency, class_frequency = class_frequency)}")

Örnek e-posta şu şekilde etiketlenmiştir: 0
Log Naive Bayes modeli bunu şu şekilde sınıflandırır: 0


Bu geliştirilmiş algoritma ile yeni doğruluk şu şekildedir:

In [63]:
# Test seti için tahminleri alalım:

# Tahminleri depolamak için boş bir liste oluştur
Y_pred = []


# Test kümesindeki her e-postayı yineleyin
for email in X_test:
    # Tahmini gerçekleştir
    prediction = log_naive_bayes(email,word_frequency = word_frequency, class_frequency = class_frequency)
    # Listeye ekle
    Y_pred.append(prediction)

# Gerçek pozitiflerin sayısını al:
true_positives = get_true_positives(Y_test, Y_pred)

# Gerçek negatiflerin sayısını al:
true_negatives = get_true_negatives(Y_test, Y_pred)

print(f"Gerçek pozitiflerin sayısı: {true_positives}\nGerçek negatiflerin sayısı: {true_negatives}")

# Doğruluğu, gerçek negatifleri gerçek pozitiflerle toplayıp veri kümesindeki toplam eleman sayısına bölerek hesaplayın.
# Hem Y_pred hem de Y_test aynı uzunluğa sahip olduğundan hangisini kullandığınız önemli değildir.
accuracy = (true_positives + true_negatives)/len(Y_test)

print(f"Doğruluk: {accuracy:.4f}")

Gerçek pozitiflerin sayısı: 249
Gerçek negatiflerin sayısı: 888
Doğruluk: 0.9921


Bu **çok büyük** bir gelişme! Modelin doğruluğunu test setinde %86,13'den %99,21'e çıkardınız! Neredeyse %13'lik bir artış. Ve veri setine dokunmadınız, bu tamamen arkasındaki **matematik** açısından bir gelişmeydi. Güçlü, değil mi?

<a name="5.2"></a>
### 5.2 Model performansını geliştirme: Naive Bayes ile pratik uygulama

#### 5.2.1 Giriş

Bu bölümde, yukarıda tanımladığınız her iki Naive Bayes modelini  bir sorunu çözmek için kullanacaksınız:

Belirli bir e-posta yazılımında çalıştırmak için iyi bir spam algılama modeli geliştirmelisiniz. Bu bölümde çalıştığınız veri kümesi, bu yazılımdan elde ettiğiniz e-posta tabanıdır. Kullanıcıları spam almaktan etkili bir şekilde korumak için bir yöntem oluşturmalısınız, **ancak spam klasörüne ham e-postalar göndermekten kaçınmalısınız** çünkü bu, bir kullanıcının önemli e-postalarını kaybetmesine neden olabilir. Öte yandan, birkaç spam e-postasının gelen kutusu klasörüne geçmesine izin vermekle ilgili değildir.

#### 5.2.2 Doğruluk ve sınırlamaları

Şu anda, şu ana kadar geliştirdiğiniz modelin gerçek performansı nedir? Yukarıda tanımladığınız doğruluk metriğinin, özellikle bu spam algılama sorununda bazı sınırlamaları vardır. Defterin başında, veri kümesindeki spam e-postalarının oranının %23,88 olduğunu gördünüz. Yani, **her e-postayı doğrudan gelen kutusu klasörüne** göndermek için bir kural oluşturursanız, her e-postanın %76,12'sini doğru şekilde sınıflandırır! Yani bu anlamsız kuralın doğruluğu %76,12'dir.

Bu soruyu doğru şekilde cevaplamaya çalışmak için kendinize iki soru sorabilirsiniz:

- Algoritma kaç tane spam e-postayı doğru şekilde spam olarak sınıflandırıyor? Bunlara **gerçek pozitifler** denir.
- Algoritma kaç tane **amatör** e-postayı **yanlışlıkla** spam olarak sınıflandırıyor? Bunlara **yanlış pozitifler** denir. **Bu, daha yakından bakmanız gereken önemli sorudur.**

İlk soru [*geri çağırma*](https://en.wikipedia.org/wiki/Precision_and_recall) adlı bir metrikle ilgilidir. İlk soruyu cevaplamak için, veri setinde kaç tane spam e-postası olduğunu saymalı ve bunlardan kaçının model tarafından doğru bir şekilde spam olarak etiketlendiğini (gerçek pozitifler) saymalısınız. Bu, geri çağırma olarak tanımlanır:

$$\text{geri çağırma} = \frac{\text{gerçek pozitifler (spam olarak doğru bir şekilde etiketlenen spam e-postalar)}}{\text{her spam e-posta}}$$

Bu metriğin tanımlanmasının bir başka yolu da, bir spam e-postasının doğru bir şekilde spam olarak etiketleneceğini (gerçek pozitif) veya yanlışlıkla ham olarak etiketleneceğini (yanlış negatif) düşünmektir, bu nedenle

$$\text{geri çağırma} =\frac{\text{gerçek pozitifler}}{\text{gerçek pozitifler} + \text{yanlış negatifler}}$$

Şimdi geri çağırma işlevini yapacaz.

In [64]:
def get_recall(Y_true, Y_pred):
    """
    İkili sınıflandırma görevinde recall (geri çağırma) değerini hesaplar.

    Parametreler:
    - Y_true (array-like): Gerçek etiketler (ground truth).
    - Y_pred (array-like): Tahmin edilen etiketler.

    Dönen Değer:
    - recall (float): Recall skoru, gerçek pozitiflerin (true positives) toplam gerçek pozitiflere (actual positives) oranıdır.
    """
    # Toplam spam e-postalarının sayısını buluyoruz. Veride spam'ler 1 olarak işaretlendiği için, Y_true dizisindeki 1'lerin toplamı bu sayıyı verir.
    total_number_spams = Y_test.sum()
    
    # Gerçek pozitifleri (true positives) hesaplıyoruz
    true_positives = get_true_positives(Y_true, Y_pred)
    
    # Recall'ı hesaplıyoruz (true_positives / toplam spam sayısı)
    recall = true_positives / total_number_spams
    return recall


In [65]:
# Naive Bayes modelini (standart ve log versiyonlarını) kullanarak test veri setindeki her bir e-postayı sınıflandırıyoruz
Y_pred_naive_bayes = []  # Standart Naive Bayes tahmin sonuçları
Y_pred_log_naive_bayes = []  # Log Naive Bayes tahmin sonuçları

# X_test'teki her e-posta için tahmin yapıyoruz
for email in X_test:
    # Naive Bayes modelini kullanarak tahmin yapıyoruz
    prediction = naive_bayes(email, word_frequency=word_frequency, class_frequency=class_frequency)
    # Log Naive Bayes modelini kullanarak tahmin yapıyoruz
    log_prediction = log_naive_bayes(email, word_frequency=word_frequency, class_frequency=class_frequency)
    
    # Her iki modelin tahmin sonuçlarını ilgili listelere ekliyoruz
    Y_pred_naive_bayes.append(prediction)
    Y_pred_log_naive_bayes.append(log_prediction)

# Her iki model için recall değerlerini hesaplıyoruz
recall_naive_bayes = get_recall(Y_test, Y_pred_naive_bayes)  # Standart Naive Bayes recall'ı
recall_log_naive_bayes = get_recall(Y_test, Y_pred_log_naive_bayes)  # Log Naive Bayes recall'ı


In [66]:
print(f"Standart Naive Bayes modelinin, spam e-postalarını doğru sınıflandırma oranı (recall): {recall_naive_bayes:.4f}")
print(f"Log Naive Bayes modelinin, spam e-postalarını doğru sınıflandırma oranı (recall): {recall_log_naive_bayes:.4f}")


Standart Naive Bayes modelinin, spam e-postalarını doğru sınıflandırma oranı (recall): 0.9803
Log Naive Bayes modelinin, spam e-postalarını doğru sınıflandırma oranı (recall): 0.9803


Tamam, her iki model de **spam'leri tespit etmede** oldukça iyi performans gösteriyor ve bunların %98'ini doğru bir şekilde tespit edebiliyor! Bu ölçüm bize modelin **duyarlılığı** hakkında bilgi veriyor. Başka bir deyişle, bu ölçüm bize modelin bir spam e-postayı tespit etmede ne kadar etkili olduğunu gösteriyor.

Şimdi ikinci soruyla karşı karşıyasınız. Bu, *[kesinlik](https://en.wikipedia.org/wiki/Precision_and_recall)* adlı başka bir ölçümle alakalı.

Soruyu cevaplamak için Naive Bayes modellerinin spam olarak sınıflandırdığı tüm e-postalara ve bu havuzda **aslında** kaç tanesinin spam olduğuna bakmalısınız. Bu, bakılması gereken önemli bir ölçümdür çünkü herhangi bir e-postayı spam olarak sınıflandıran bir model, spam e-postaların %100'ünü doğru bir şekilde sınıflandıran bir modeldir, ancak anlamsızdır! Ayrıca, spam klasörüne düzenli e-postalar göndermekten kaçınmalısınız, aksi takdirde kullanıcılar önemli e-postaları kaybedebilir.

Bu soru **yanlış pozitifler** olarak adlandırılan şeyle ilgilidir. Başka bir deyişle, şimdi algoritmanın spam klasörüne kaç tane ham e-posta gönderdiğine bakıyorsunuz. Bir sonraki kod bloğunda, yanlış pozitifleri hesaplamak için bir fonksiyon oluşturacaz.

In [67]:
def get_false_positives(Y_true, Y_pred):
    """
    İkili sınıflandırma problemlerinde, yanlış pozitif (false positive) sayısını hesaplar.

    Parametreler:
    - Y_true (list): Her bir örneğin gerçek etiketleri (0 veya 1).
    - Y_pred (list): Her bir örneğin tahmin edilen etiketleri (0 veya 1).

    Döndürülen değer:
    - int: Yanlış pozitif sayısı, gerçek etiketin 0 ve tahmin edilen etiketin 1 olduğu durumlar.
    """
    
    # Y_true ve Y_pred listelerinin uzunluklarının eşit olduğundan emin olunmalıdır.
    if len(Y_true) != len(Y_pred):
        return "Gerçek etiket sayısı ile tahmin edilen etiket sayısı eşleşmelidir!"
    
    n = len(Y_true)  # Y_true'nun uzunluğu

    false_positives = 0  # Yanlış pozitif sayısını sıfırla başlatıyoruz
    # Liste elemanları üzerinde dönüyoruz
    for i in range(n):
        # Şu anki e-posta için gerçek etiketi alıyoruz
        true_label_i = Y_true[i]
        # Şu anki e-posta için tahmin edilen etiketi alıyoruz
        predicted_label_i = Y_pred[i]
        # Yalnızca gerçek etiket 0 ve tahmin edilen etiket 1 olduğunda, yanlış pozitif sayısını 1 artırıyoruz
        if true_label_i == 0 and predicted_label_i == 1:
            false_positives += 1
    
    return false_positives  # Yanlış pozitif sayısını geri döndürüyoruz


In [68]:
# Yanlışlıkla spam olarak etiketlenen ham e-postaları sayalım (false positives). Yukarıda gördüğümüz get_false_positives fonksiyonunu kullanalım

false_positives_naive_bayes = get_false_positives(Y_test, Y_pred_naive_bayes)  # Standart Naive Bayes modelinin yanlış pozitiflerini hesapla
false_positives_log_naive_bayes = get_false_positives(Y_test, Y_pred_log_naive_bayes)  # Log Naive Bayes modelinin yanlış pozitiflerini hesapla


In [69]:
print(f"Standart Naive Bayes modelindeki yanlış pozitif sayısı: {false_positives_naive_bayes}")
print(f"Log Naive Bayes modelindeki yanlış pozitif sayısı: {false_positives_log_naive_bayes}")


Standart Naive Bayes modelindeki yanlış pozitif sayısı: 154
Log Naive Bayes modelindeki yanlış pozitif sayısı: 4


Bu çok büyük bir gelişme! 169 adet ham e-postanın yanlışlıkla spam olarak etiketlenmesinden sadece 4'e düştünüz! Daha anlamlı bir sayı elde etmek için aşağıdaki miktarı hesaplayabilirsiniz:

- Tahmin edilen spam e-posta havuzunda bulunan gerçek spam e-postalarının (gerçek pozitifler) oranı. Tahmin edilen e-posta havuzunun **doğru şekilde spam olarak etiketlenen her spam e-postasından** (gerçek pozitifler) ve **yanlışlıkla spam olarak etiketlenen her ham e-postasından** (yanlış pozitifler) oluştuğunu unutmayın.

Bu miktara **kesinlik** denir ve şu şekilde tanımlanır:

$$\text{kesinlik} = \frac{\text{gerçek pozitifler}}{\text{gerçek pozitifler} + \text{yanlış pozitifler}}$$

Bu ölçüm, modelinizin çıktısının ne kadar **ilgili** olduğunu söyler. Daha önce tartışıldığı gibi, her e-postayı spam olarak öngören bir model her spam e-postayı doğru bir şekilde tanımlayabilir, ancak her ham e-postayı spam klasörüne gönderdiği için çıktısı önemsizdir. Şimdi bunu uygulayacaz.

In [70]:
def get_precision(Y_true, Y_pred):
    """
    Bir sınıflandırma modelinin performansını ölçmek için hassasiyet (precision) hesaplar.
    Hassasiyet, doğru pozitiflerin, doğru pozitifler ve yanlış pozitiflerin toplamına oranıdır.

    Parametreler:
    - Y_true (liste): Gerçek etiketler.
    - Y_pred (liste): Tahmin edilen etiketler.

    Dönen:
    - precision (float): Hesaplanan hassasiyet skoru.
    """
    # Doğru pozitifleri al
    true_positives = get_true_positives(Y_true, Y_pred)
    # Yanlış pozitifleri al
    false_positives = get_false_positives(Y_true, Y_pred)
    # Hassasiyeti hesapla
    precision = true_positives / (true_positives + false_positives)
    return precision


In [71]:
print(f"Standart Naive Bayes modelinin hassasiyeti: {get_precision(Y_test, Y_pred_naive_bayes):.4f}")
print(f"Log Naive Bayes modelinin hassasiyeti: {get_precision(Y_test, Y_pred_log_naive_bayes):.4f}")


Standart Naive Bayes modelinin hassasiyeti: 0.6179
Log Naive Bayes modelinin hassasiyeti: 0.9842


Modelin ilk versiyonu %59,57 hassasiyete sahip. Başka bir deyişle, modelin spam olarak sınıflandırdığı 100 e-postadan yalnızca yaklaşık 60'ı gerçekten spam. Bu, bu modelin spam klasörüne 40 amatör e-posta göndereceği anlamına geliyor, bu da çok hassas olmasına rağmen çok güvenilir olmadığını gösteriyor.

Öte yandan, geliştirilmiş modelin hassasiyeti %98,42! Yani spam olarak sınıflandırılan 100 e-postadan yalnızca yaklaşık 2'si gerçekten amatör e-posta olacak. Çok daha güvenilir bir çıktı.